# 决策树


# 多决策树 - 随机森林 - 随机决策
对决策树的一个常见批评是，一旦在回答问题后对训练集进行划分，就不可能重新考虑这个决策。例如，如果我们将男性和女性分开，那么每个后续问题都只涉及男性或女性，而且该方法不能考虑其他类型的问题（例如，年龄不到一岁，不论性别如何）。**随机森林**尝试在每个步骤中引入一定程度的**随机化**，创建备选树并将它们组合来获得最终预测。考虑几个回答相同问题的分类器的这些类型的算法，被称为**集成方法**.   
随机森林建议基于训练实例的子集（**带放回随机选择**）来构建决策树，但是在特征集的每个集合中使用少量随机的特征。这种树生长过程重复几次，产生一组分类器。在预测时，给定一个实例的每个成型的树都会像决策树一样预测其目标类。大多数树所投票的类（即树中预测最多的类）是集成分类器所建议的类。  
随机森林只是许多树，建立在数据的不同随机子集（带放回抽样）上，并对于每个分裂，使用特征的不同随机子集（无放回抽样）。 这使得树彼此不同，并使它们过拟合不同的方面。 然后，他们的预测被平均，产生更平稳的估计，更少过拟合。

## sk-learn 官方User Guide的例子 
DecisionTreeClassifier 是能够在数据集上执行多分类的类,与其他分类器一样，DecisionTreeClassifier 采用输入两个数组：数组X，用 [n_samples, n_features] 的方式来存放训练样本。整数值数组Y，用 [n_samples] 来保存训练样本的类标签:

In [ ]:
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_selection import SelectFromModel
from sklearn import metrics
import matplotlib.pyplot as plt
import graphviz
from sklearn import datasets
import pandas as pd
import numpy as np
np.set_printoptions(precision=4, suppress=True, threshold=15)
pd.options.display.max_rows = 20

In [ ]:
iris = datasets.load_iris()
X, y = iris.data, iris.target
dt_clf = DecisionTreeClassifier()
dt_clf.fit(X, y)

In [ ]:
cross_val_score(dt_clf, iris.data, iris.target, cv=10)

经过训练，我们可以使用 export_graphviz 导出器以 Graphviz 格式导出决策树.

In [ ]:
dot_data = export_graphviz(dt_clf, out_file=None)
graph = graphviz.Source(dot_data)
graph
# graph.render("iris", format='png')  保存成其他格式

export_graphviz 还支持各种美化，包括通过他们的类着色节点（或回归值），如果需要，还能使用显式变量和类名

In [ ]:
dot_data = export_graphviz(dt_clf, out_file=None,
                        feature_names=iris.feature_names,
                        class_names=iris.target_names,
                        filled=True, rounded=True,
                        special_characters=True  # 忽略特殊字符
                        )
graph = graphviz.Source(dot_data)
graph

[实用技巧](https://github.com/apachecn/sklearn-doc-zh/blob/master/docs/0.21.3/11.md#1105-%E5%AE%9E%E9%99%85%E4%BD%BF%E7%94%A8%E6%8A%80%E5%B7%A7)

[数据科学和人工智能技术笔记 十三、树和森林](https://github.com/apachecn/ds-ai-tech-notes/blob/master/13.md)

In [ ]:
# 使用随机森林
clf = RandomForestClassifier(random_state=12345, n_jobs=-1)
clf.fit(X, y)

**特征的重要性**

In [ ]:
# 计算特征重要性  越接近于1 表示越重要
importances = clf.feature_importances_  # 所有重要性得分加起来为 100%
importances  

In [ ]:
# 整个数据集上的特征重要性分布
plt.bar(range(X.shape[1]), importances)
plt.title('Feature Importamces')
plt.xticks(range(X.shape[1]), iris.feature_names, rotation=90)

**使用随机森林的特征选择**

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)

In [ ]:
clf = RandomForestClassifier(n_estimators=10000, random_state=0, n_jobs=-1)
clf.fit(X_train, y_train)

In [ ]:
# 使用4个特征进行分类的 准确率
y_pred = clf.predict(X_test)
metrics.accuracy_score(y_test, y_pred)

In [ ]:
# 创建一个选择器对象，
# 该对象将使用随机森林分类器来标识重要性大于 0.15的特征
sfm = SelectFromModel(clf, threshold=0.15)
sfm.fit(X_train, y_train)
X_important_train = sfm.transform(X_train)
X_important_test = sfm.transform(X_test)
X_important

In [ ]:
sfm.get_support(indices=True)

In [ ]:
important_names = np.array(iris.feature_names)[sfm.get_support()]
important_names  # 最重要的2个特征

In [ ]:
# 使用最重要的特征 训练随机森林
clf_important = RandomForestClassifier(n_estimators=10000, random_state=0, n_jobs=-1)
clf_important.fit(X_important_train, y_train)
# 使用2个特征 的模型准确率
y_pred = clf_important.predict(X_important_test)
metrics.accuracy_score(y_test, y_pred)

**在随机森林中处理不平衡类别**

In [ ]:
# 通过移除前 40 个观测，生成高度不平衡的类别
X = X[30:, ]
y = y[30:]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)

其中类自动加权，与它们在数据中出现的频率成反比  
$$w_j = \frac {n}{kn_j}$$
$w_j$是$j$类的权重, $n$是总观测数, $n_j$是类$j$的观测数, $k$为类的总数

In [ ]:
clf = RandomForestClassifier(random_state=0, n_jobs=-1, class_weight='balanced')
clf.fit(X_train, y_train)

In [ ]:
y_pred = clf.predict(X_test)
metrics.accuracy_score(y_test, y_pred)

## [sklearn-cookbook 例子](https://github.com/apachecn/sklearn-cookbook-zh/blob/master/4.md#41-%E4%BD%BF%E7%94%A8%E5%86%B3%E7%AD%96%E6%A0%91%E5%AE%9E%E7%8E%B0%E5%9F%BA%E6%9C%AC%E7%9A%84%E5%88%86%E7%B1%BB)

In [ ]:
# 生成n-class(默认2个)的样本
# 3特征 其中0冗余 0重复
X, y = datasets.make_classification(n_samples=1000, n_features=3, n_redundant=0)
X

In [ ]:
dt = DecisionTreeClassifier()
dt.fit(X, y)

In [ ]:
preds = dt.predict(X)
np.mean(preds == y)

**max_depth** 决策树最大深度, 决定了分支的数量

In [ ]:
n_features = 200
X, y = datasets.make_classification(1000, n_features=n_features, n_informative=5)  # 有用的特征数5个
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)
accuracies = [] # 保存正确率

In [ ]:
# 不同的最大深度 的影响
for x in range(1, n_features+1):
    dt_clf = DecisionTreeClassifier(max_depth=x)
    dt_clf.fit(X_train, y_train)
    score = dt_clf.score(X_test, y_test)
    accuracies.append(score)

In [ ]:
np.argmax(accuracies) + 1

In [ ]:
# 可视化处理
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
plt.plot(range(1, n_features+1), accuracies)
plt.xlabel('Max Depth')
plt.ylabel('Score')

实际上在较低最大深度处得到了漂亮的准确率.

In [ ]:
N = 15
plt.plot(range(1, n_features+1)[:N], accuracies[:N])
plt.xlabel('Max Depth')
plt.ylabel('Score')

**调整决策树模型**

In [ ]:
X, y = datasets.make_classification(1000, n_features=20, n_informative=3)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)
def dt_max_depth(*args, **kwargs):
    dt = DecisionTreeClassifier(*args, **kwargs)
    dt.fit(X_train, y_train)
    return dt

In [ ]:
def view_dt(dt):
    dot_data = export_graphviz(dt, out_file=None)
    graph = graphviz.Source(dot_data)
    return graph

In [ ]:
view_dt(dt_max_depth())  # 10层 

In [ ]:
# 降低最大深度
view_dt(dt_max_depth(max_depth=5)) 

In [ ]:
# 使用信息增益(熵)进行分割
view_dt(dt_max_depth(max_depth=5, criterion='entropy', min_samples_leaf=10))   # 叶结点最小样本数10

##  用决策树解释泰坦尼克号假设
[sklearn 学习指南](https://github.com/apachecn/misc-docs-zh/blob/master/docs/learning-sklearn/ch02.md#%E7%94%A8%E5%86%B3%E7%AD%96%E6%A0%91%E8%A7%A3%E9%87%8A%E6%B3%B0%E5%9D%A6%E5%B0%BC%E5%85%8B%E5%8F%B7%E5%81%87%E8%AE%BE)

属性列表为：Ordinal（序号），Class（等级），Survived（是否幸存，0=no，1=yes），Name（名称），Age（年龄），Port of Embarkation（登船港口），Home/Destination（家/目的地），Room（房间），Ticket（票号），Boat（救生艇）和Sex（性别）

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
data = pd.read_csv('titanic')
data

### 预处理数据   
选择 pclass age sex 这3种数据进行划分

In [ ]:
titanic_X , titanic_y = data.iloc[:, [1, 4, -1]], data.iloc[:, 2]
features = titanic_X.columns
features

In [ ]:
titanic_y.value_counts()

**处理缺失值**

将年龄的缺失值用所有人员年龄的平均值进行替换

In [ ]:
titanic_X = titanic_X.fillna(titanic_X.mean())
X = titanic_X.values  # pandas -> array
X

In [ ]:
y = titanic_y.values
y

**类别特征编码**  
将标签值转为0..K-1的整数
```
class sklearn.preprocessing.LabelEncoder
    Encode labels with value between 0 and nclasses
```

In [ ]:
from sklearn import preprocessing
enc = preprocessing.LabelEncoder()
X[:, -1] = enc.fit_transform(X[:, -1])  # 直接对 性别 这一列进行转换
X

In [ ]:
enc.classes_ 

对 pclass 进行这样处理,会有3个结果 0, 1, 2.这种转换隐式地引入了类之间的顺序, 但实际却是无序的.  
另外一种将标称型特征转换为能够被scikit-learn中模型使用的编码是one-of-K， 又称为**独热码**或dummy encoding。  
这种编码类型已经在类`OneHotEncoder`中实现。该类把每一个具有n_categories个可能取值的categorical特征变换为长度为n_categories的二进制特征向量，里面只有一个地方是1，其余位置都是0.  
1st 2nd 3rd 

In [ ]:
# 将整数特征 变 独热码
enc_pclass = preprocessing.OneHotEncoder()
enc_pclass

In [ ]:
new_pclass = enc_pclass.fit_transform(X[:,0][:, np.newaxis]).toarray()
new_pclass  # 将pclass列 n个数据变成 n个特征向量, 每个向量中只有一个为1 其余为0

In [ ]:
X

In [ ]:
X = np.concatenate((new_pclass, X[:, 1:]), axis=1)  # 最终得到的数据
X

In [ ]:
enc_pclass.get_feature_names()

In [ ]:
enc_pclass.categories_[0]

In [ ]:
new_features = np.append(enc_pclass.categories_[0], np.array(features[1:]))
new_features

### 训练决策树

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=12345)

In [ ]:
dt = DecisionTreeClassifier(criterion='entropy', max_depth=3, min_samples_leaf=5)
dt.fit(X_train, y_train)

In [ ]:
dot_data = export_graphviz(dt, out_file=None, feature_names=new_features)
graphviz.Source(dot_data)

### 评估分类器

In [ ]:
def measure_performance(X, y, clf):
    y_pred = dt.predict(X)
    print(f"Accuracy: {metrics.accuracy_score(y, y_pred):.3f}")  #  精确度得分
    print('Classification report: ')
    print(metrics.classification_report(y, y_pred))  # 具体的分类指标
    print("Confussion matrix")
    """ 从混淆矩阵的迹除以总和来计算准确度
    TN(真阴)  FP(假阳)
    FN(假阴)  TP(真阳)
    """
    print(metrics.confusion_matrix(y, y_pred))

In [ ]:
measure_performance(X_train, y_train, dt)

In [ ]:
measure_performance(X_test, y_test, dt)

**留一交叉验证(LeaveOneOut, LOO)**  
每个学习集都是通过除了一个样本以外的所有样本创建的，测试集是被留下的样本。 因此，对于 n 个样本，我们有 n 个不同的训练集和 n 个不同的测试集

In [ ]:
from sklearn.model_selection import LeaveOneOut
from scipy.stats import sem   # 计算平均值的标准误差
def loo_cv(X, y, clf):
    scores = np.zeros(X.shape[0])
    loo = LeaveOneOut()
    # test_index 长度为1
    for train_index, test_index in loo.split(X):  # [1, 2, 3,...] [0]
        # print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        clf = clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        scores[test_index] = metrics.accuracy_score(y_test, y_pred)
    print(f"Mean score: {np.mean(scores):.3f}(+/-{sem(scores):.3f})")

In [ ]:
loo_cv(X_train, y_train, dt)

留一法交叉验证的主要优点是它允许训练的数据几乎与我们可用的数据一样多，因此它特别适用于数据稀缺的情况。其主要问题是，就计算时间而言，为每个实例训练不同的分类器可能是非常昂贵的。

In [ ]:

rf_clf = RandomForestClassifier(n_estimators=10, random_state=33, n_jobs=-1)
rf_clf.fit(X_train, y_train)

In [ ]:
rf_clf.n_features_  # The number of features when fit is performed.

In [ ]:
measure_performance(X_train, y_train, rf_clf)

In [ ]:
measure_performance(X_test, y_test, rf_clf)

In [ ]:
loo_cv(X_train, y_train, rf_clf)

随机森林的结果实际上更糟。毕竟，引入随机化似乎不是一个好主意，因为特征数量太少。然而，对于具有更多特征的更大数据集，随机森林是一种非常快速，简单且流行的方法，可以提高准确率，保留决策树的优点